## Start

In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import glob
from datetime import datetime
from scipy import optimize

In [2]:
project_path = Path.cwd().parent

## math_utils

### calculate_metrics

In [3]:
from support_functions.data_loader import load_data
from support_functions.flow_builders import build_stock_cash_flows, build_account_cash_flows
from support_functions.math_utils import xirr

In [4]:
data_dir = f'{project_path}/data'
output_dir = f'{project_path}/output'

data = load_data(data_dir)

account = 'Z23390746'
symbol = 'AAPL'
entity_cash_flows = build_stock_cash_flows(data, account, symbol)

Loading positions from: /Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv (Date: 2025-12-31)
Found 4 history files.


In [5]:
cash_flows = entity_cash_flows.cash_flows
total_invested = entity_cash_flows.total_invested
current_value = entity_cash_flows.current_value
latest_date = entity_cash_flows.latest_date

In [6]:
irr_val = xirr(cash_flows)
irr_val

np.float64(0.19542017278306092)

In [7]:
total_return_dollar = current_value - total_invested
total_return_dollar

np.float64(8270.210000000001)

In [8]:
roi = total_return_dollar / total_invested if total_invested != 0 else 0
roi

np.float64(0.7686075916427433)

In [ ]:
invested_cash_flows = [flow for flow in cash_flows if flow[1] < 0]

dates, amounts = zip(*invested_cash_flows)
amounts = pd.Series(amounts)
amounts_ratio = amounts/amounts.sum()
holding_period = pd.Series([(latest_date - d).days for d in dates])
(amounts_ratio*holding_period).sum()/365.0

np.float64(3.2792546033710095)

In [24]:
holding_period

0    1244
1    1244
2    1223
3    1220
4    1156
5    1149
dtype: int64

## analysis

### analyze_account_performance

In [27]:
from support_functions.data_loader import load_data
from support_functions.flow_builders import build_stock_cash_flows, build_account_cash_flows
from support_functions.math_utils import calculate_metrics

In [30]:
data_dir = f'{project_path}/data'
output_dir = f'{project_path}/output'

data = load_data(data_dir)

Loading positions from: /Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv (Date: 2025-12-31)
Found 4 history files.


In [34]:
positions = data.positions
transactions = data.transactions
latest_date = data.latest_date


In [35]:
accounts_unique = positions.groupby(['Account Number','Account Name'])['Current Value'].sum().reset_index()

In [38]:
row = accounts_unique.iloc[0]
account = row['Account Number']
entity_cash_flows = build_account_cash_flows(data, account)
metrics = calculate_metrics(entity_cash_flows)

In [ ]:
account = 'Z23390746'
symbol = 'AAPL'
entity_cash_flows = build_stock_cash_flows(data, account, symbol)

,Account Number,Account Name,Current Value
0,241802439,Health Savings Account,9707.44
1,86964,ERNST & YOUNG 401(K),29217.10
2,Z06872898,Cash Management (Individual),108.17
3,Z23390746,Individual,1250701.91


In [39]:
metrics

{'IRR': np.float64(0.14987364529865643),
 'Total Return ($)': np.float64(1256.7999999999956),
 'ROI': np.float64(0.1487224636240563)}